In [1]:
!git clone https://github.com/FaridRash/Advance-Statistical-Learnin--Semester-3

Cloning into 'Advance-Statistical-Learnin--Semester-3'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 44 (delta 9), reused 11 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (44/44), 33.01 MiB | 15.53 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
data_path = "/kaggle/working/Advance-Statistical-Learnin--Semester-3/Dataset/EURUSD-2000-2020-15m.csv"
df = pd.read_csv(data_path)

In [6]:

def EMA(df, base, target, period, alpha=False):
    """
    Function to compute Exponential Moving Average (EMA)
    """
    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
    if (alpha == False):
        df[target] = con.ewm(span=period, adjust=False).mean()
    else:
        df[target] = con.ewm(alpha=alpha, adjust=False).mean()
    return df

df_ema = EMA(df, 'CLOSE', 'EMA_20', 20)

In [7]:
df_ema.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,DATE_TIME,HIGH,LOW,OPEN,CLOSE,EMA_20
0,2000.01.03 00:00:00,1.0080,1.0073,1.0073,1.0077,NaN
1,2000.01.03 00:15:00,1.0087,1.0076,1.0078,1.0086,NaN
2,2000.01.03 00:30:00,1.0089,1.0079,1.0087,1.0079,NaN
3,2000.01.03 00:45:00,1.0132,1.0078,1.0078,1.0128,NaN
4,2000.01.03 01:00:00,1.0133,1.0120,1.0129,1.0122,NaN


#Packing

In [8]:
def create_packs(df, pack_size):

    numerical_data = df[['HIGH', 'LOW', 'OPEN', 'CLOSE', 'EMA_20']].values
    
    num_packs = len(numerical_data) - pack_size + 1
    
    packs = np.array([numerical_data[i:i + pack_size] for i in range(num_packs)])
    
    return packs

In [9]:
pack_size = 100 
packs = create_packs(df_ema, pack_size)

In [10]:
type(packs), packs.shape

(numpy.ndarray, (500652, 100, 5))

#Normalization

In [11]:
def normalize_packs(packs):

    scaler = MinMaxScaler()
    
    num_packs, pack_size, num_features = packs.shape
    reshaped_packs = packs.reshape(-1, num_features)
    
    normalized_data = scaler.fit_transform(reshaped_packs)
    
    normalized_packs = normalized_data.reshape(num_packs, pack_size, num_features)
    
    return normalized_packs


In [12]:
normalized_packs = normalize_packs(packs)

print("Shape of normalized packs:", normalized_packs.shape)

Shape of normalized packs: (500652, 100, 5)


In [13]:
train_size = 0.7
val_size = 0.1
test_size = 0.2

packs_train, packs_temp, norm_packs_train, norm_packs_temp = train_test_split(
    packs, normalized_packs, test_size=(val_size + test_size), random_state=42)

In [14]:
val_ratio = val_size / (val_size + test_size)  # Adjust ratio for temp split
packs_val, packs_test, norm_packs_val, norm_packs_test = train_test_split(
    packs_temp, norm_packs_temp, test_size=(1 - val_ratio), random_state=42
)

In [15]:
print("Packs Train size:", len(packs_train))
print("Packs Validation size:", len(packs_val))
print("Packs Test size:", len(packs_test))
print('-----------')
print("Normalized Train size:", len(norm_packs_train))
print("Normalized Validation size:", len(norm_packs_val))
print("Normalized Test size:", len(norm_packs_test))

Packs Train size: 350456
Packs Validation size: 50065
Packs Test size: 100131
-----------
Normalized Train size: 350456
Normalized Validation size: 50065
Normalized Test size: 100131


In [23]:
def get_state(packs, index):

    state = packs[index]
    next_state = packs[index + 1] if index + 1 < len(packs) else None
    return state, next_state


In [24]:
def select_action(q_values, epsilon):

    if np.random.rand() < epsilon:
        action = np.random.choice([0, 1, 2])
    else:
        action = np.argmax(q_values)
    return action

In [25]:
def calculate_reward(action, open_price, stop_loss, target, target_multiplier, close_price, low, high, max_hold, current_hold):

    if action == 'buy':
        if low <= stop_loss: 
            return -1.0 
        elif high >= target:
            return target_multiplier

    elif action == 'sell':
        if high >= stop_loss: 
            return -1.0
        elif low <= target: 
            return target_multiplier

    elif current_hold >= max_hold:
        return (close_price - open_price) / open_price - 0.1 

    elif action == 'hold':
        return -0.05

    # Default: No action has triggered any event yet
    return 0.0

In [28]:
class ForexTradingEnv:
    def __init__(self, packs, max_hold):
        self.packs = packs
        self.max_hold = max_hold
        self.current_index = 0
        self.current_hold = 0
        self.open_price = None
        self.stop_loss = None
        self.target = None
        self.balance = 1000  # Starting balance
        self.done = False

    def reset(self):
        """Reset the environment to the initial state."""
        self.current_index = 0
        self.current_hold = 0
        self.open_price = None
        self.stop_loss = None
        self.target = None
        self.balance = 1000  # Reset balance
        self.done = False
        # Correct call to `get_state` with both arguments
        state, _ = get_state(self.packs, self.current_index)
        return state

    def step(self, action):
        """Take an action and transition to the next state."""
        if self.done:
            raise ValueError("Cannot step in a finished environment. Call reset().")

        # Current state
        current_state, _ = get_state(self.packs, self.current_index)

        # Transition to the next state
        result = transition_to_next_state(
            current_state=current_state,
            action=action,
            current_hold=self.current_hold,
            packs=self.packs,
            pack_index=self.current_index,
            open_price=self.open_price,
            stop_loss=self.stop_loss,
            target=self.target,
            max_hold=self.max_hold,
            balance=self.balance,
        )

        next_state, reward, self.open_price, self.stop_loss, self.target, self.current_hold, self.balance, self.done = result

        # Increment index if not done
        if not self.done:
            self.current_index += 1

        return next_state, reward, self.done

    def render(self):
        """Render the current environment state."""
        print(f"Index: {self.current_index}")
        print(f"Balance: {self.balance}")
        print(f"Open Price: {self.open_price}")
        print(f"Stop Loss: {self.stop_loss}")
        print(f"Target: {self.target}")
        print(f"Current Hold: {self.current_hold}")


In [27]:
env = ForexTradingEnv(packs=packs, max_hold=10)
state = env.reset()
done = False
while not done:
    action = select_action(state)  # Replace with your action selection logic
    next_state, reward, done = env.step(action)
    env.render()


TypeError: get_state() missing 1 required positional argument: 'index'